In [1]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
import re 
from tqdm.notebook import tqdm
import random

import shutil #이미지저장

### get urls  많이 하면 인스타그램에서 로봇으로 판정하는듯함.
* 해결방안은 있을까? -> 휴식시간을 많이주니까 걸리진 않은듯

### 통합
* 2021-01-19 기본 쉬는시간 : 6 + 0.7X100 + (3~5)X100, 800건 기준 18시간 걸림(too_late)
* 2021-01-20 기본 쉬는시간 : 4 + 0.5X1000 + (2~4)X1000, 미정 (없어질 각오하고 크게돌리기)

In [ ]:
#크롬드라이버 작동
user = 'id'
password = 'pw'
keyword = "풀빌라" 
before_data= '풀빌라_822.csv'
scroll_num = 1000 #스크롤 횟수 많을수록 검색 url수 증가 100기준 700건
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
before = True
img_save = True
url = "https://www.instagram.com/explore/tags/{}/".format(keyword) 
# 크롬드라이버 작동 
options = Options()
options.add_argument('--start-maximized') #전체화면
options.add_argument(f'user-agent={headers}')
options.add_argument('--headless') #headless(화면안켜고)
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)
driver.get(url) 
time.sleep(2) 

try : #로그인 안해도 되는 경우도 있음
    #로그인(id, password 입력)
    ele = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input')
    ele.send_keys(user)
    ele = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(2) > div > label > input')
    ele.send_keys(password)
    ele = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button > div')
    ele.click()
    time.sleep(2)
    ele = driver.find_element_by_css_selector('#react-root > section > main > div > div > div > div > button') #로그인저장 창 통제
    ele.click()
except :
    pass

#url 수집
url_halfs = []
count = 1
for i in tqdm(range(0, scroll_num)): 
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    #url저장
    url_source = soup.select('#react-root > section > main > article  a')
    for url_half in url_source:
        url_halfs.append(url_half.attrs['href'])
        
    #중복제거
    url_halfs = list(set(url_halfs))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #휠스크롤
    time.sleep(0.5)
    count =+ i
#     print(count)
print(f'수집된 url수:{len(url_halfs)}')
driver.close()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


In [ ]:
#이전 url이 있는 경우 제외

b_url = pd.read_csv(before_data)['url']
b_url = b_url.apply(lambda x : x.replace('https://www.instagram.com/',''))

url_s = pd.Series(url_halfs)
url_halfs = list(url_s[~(url_s.apply(lambda x : any([url in x for url in b_url])))].values)

#드라이버 재시작
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#각 게시글에서 정보추출
nonetype = type(print()) #타입비교 용도
index_dict = dict() #데이터 임시저장
if img_save == True:
    os.makedirs('./img',exist_ok=True) #이미지_저장폴더

for index, url_half in tqdm(enumerate(url_halfs)):
    #저장 초기화
    contents = dict()
    contents['dates'] = []
    contents['user_id'] = []
    contents['body'] = []
    contents['hashtags'] = []
    contents['comments'] = []
    contents['like'] = []
    contents['movie'] = []
    contents['url'] = []
    contents['image'] =[]
    real_url = f'https://www.instagram.com/{url_half}'
    driver.get(real_url)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    #사람처럼 행동 random하게 스크롤 작동
    ran_cnt = random.randint(1,5)
    ran_length = random.choice(np.arange(100,200,100))
    for count in range(0,ran_cnt):
        driver.execute_script(f"window.scrollTo(0, {ran_length})")
        ran_length *= 2
#         time.sleep(0.5)
    
    #dates
    try :
        dates = soup.select('div.k_Q0X.NnvRN')[0].text
    except :
        dates = ''
    
    #id
    try :
        user_id = soup.select('span.Jv7Aj.mArmR.MqpiF')[0].text
    except :
        user_id = ''
    
    #본문
    try :
        body = soup.select_one('body div.C4VMK > span').text
    except : 
        body = []
    
    #comment
    try : 
        comments = soup.select('div.C4VMK > span')
        comments = [comment.text for comment in comments]
    except:
        comments = []
        
    #해쉬태그
    try :
        hashtags = soup.find_all(attrs={'class':'xil3i'})
        hashtags = [hashtag.text for hashtag in hashtags]
    except :
        print('no hashtag')
        hashtags.append([])

    #좋아요수(동영상의 경우 view) #코드 요약 필요할듯 
    p_tag = soup.select_one('div.Nm9Fw') #사진테그
    m_tag = soup.select_one('span.vcOH2') #동영상테그
    if type(m_tag) == nonetype: #사진인경우
#         print('p_tag')
        try:
            movie=0
            like = soup.select_one('div.Nm9Fw span').text #좋아요 수         
        except :
            like = 0
    elif type(p_tag) == nonetype:#동영상인경우
#         print('m_tag')
        try :
            movie=1
            like = soup.select_one('div.vcOH2 span').text #조회수 수         
        except :
            like = 0
    #최소화해놓으면 안되고 최소화 하지 않으면 작동안함()
#     random_stop = random.randint(2,4) #인간답게하려고 랜덤으로 2~4초 스탑
#     time.sleep(random_stop)
    if img_save == True:
        try :
            imgurl = soup.select('div.KL4Bh')[0].img['src']
            response = requests.get(imgurl, stream = True)
            local_file = open(f'./img/{keyword}_{index}.jpg','wb') #저장임시파일
            response.raw.decode_content = True #역할이...?
            shutil.copyfileobj(response.raw, local_file)
#             print(f'{index}, 이미지저장')
            image = 1
        except Exception as ex:
#             print(f'{index}, 이미지저장실패')
#             print(ex) 
            image = 0
            pass
      
    #저장
    contents['body'] = body
    contents['comments'] = contents['comments'] + comments
    contents['hashtags'] = contents['hashtags'] + hashtags
    contents['dates'] = dates
    contents['user_id'] = user_id
    contents['like'] = like
    contents['movie'] = movie
    contents['url'] = real_url
    contents['image'] = image
    
    #벤방지
    time.sleep(random_stop) 
    
    #인덱스맞춰저장
    index_dict[index] = contents

driver.close()
#데이터프레임화
temp = pd.DataFrame(index_dict)
temp = temp.T

### 데이터정비

#### 결측제거

In [ ]:
#결측제거
temp = temp.drop(temp[temp['dates'] == ''].index)
temp = temp.reset_index()

#### 해시테그 통합

In [ ]:
#body, comment
def get_hashtags(x,style):
    pattern = '#[0-9a-zA-Z가-힣]+'
    if style == 'body':
        tags = re.findall(pattern,x[0])
        tags = [tag.replace('#','').strip() for tag in tags]
    if style == 'hashtag':
        tags = [tag.replace('#','') for tag in x]
    return tags

body_tag = temp['body'].apply(lambda x : get_hashtags(x,'body'))
hashtag_tag = temp['hashtags'].apply(lambda x : get_hashtags(x,'hashtag'))

body_tag = body_tag.to_dict()
hashtag_tag = hashtag_tag.to_dict()

#comment
comment_tag = dict()
pattern = '#[0-9a-zA-Z가-힣]+'
for i in range(0,len(temp)):
    b_tag = []
    try : 
        for m in range(0,2):
            comment = temp['comments'].iloc[i][m]
            c_tag = re.findall(pattern,comment)
            cc_tag = [tag.replace('#','').strip() for tag in c_tag]
            b_tag = b_tag + cc_tag
    except :
        pass
    comment_tag[i] = b_tag

#모든 해시태그
all_tag = dict()

for num, i in enumerate(range(0, len(comment_tag))):
    all_tag[num] = ",".join(list(set(body_tag[i] + hashtag_tag[i] + comment_tag[i])))

temp['all_tag'] = all_tag.values()

### 날짜정리

In [ ]:
def time_change(x,nomalize = False):
    import re
    from datetime import timedelta 
    '''검색결과에 있는 문자를 날짜로 변경
    크롤링시 작성일자가 1분 전/1시간 전/1일 전으로 나타는 경우가 존재
    이러한 패턴을 일반적인 YY-MM-DD형태로 변환
    그리고 월별 계산이 중요하므로 모든 데이터는 1일로 통일
    
    Return
    -------
    x : datetime
        날짜형형태로 전환
        
    Example
    -------
    >>> x = "47분 전"
    >>> time_change(x)
    datetime.datetime(2020, 6, 1, 0, 0)
    
    >>> x2 = "2019.01.31"
    >>> time_change(x2)
    datetime.datetime(2019, 1, 1, 0, 0)
    '''
    
    number = re.compile('[0-9]+')
    today = datetime.datetime.today().date()
    x = str(x)
    # 일자
    if "일 전" in x :
        d = re.findall(number, x)[0]
        x = today - timedelta(days=int(d))
    elif "시간 전" in x:
        d = re.findall(number, x)[0]
        x = today
    elif "분 전" in x:
        d = re.findall(number, x)[0]
        x = today
    elif x == "어제":
        x = today - timedelta(days=1)
    elif "월" in x: # '%Y년 %m월 %d일'형태
        try :
            x = datetime.datetime.strptime(x, '%Y년 %m월 %d일')
        except :
            x = datetime.datetime.strptime(f'2021년 {x}', '%Y년 %m월 %d일')
    else :
        x = x
    
    #모든일자를 1일로 통일
    if nomalize:
        x = datetime.datetime.strptime(x.strftime('%Y-%m')+"-01","%Y-%m-%d")
    return x

In [ ]:
temp['dates']  = temp['dates'].apply(lambda x : time_change(x))

### 저장

In [ ]:
temp.to_csv(f'{keyword}_{len(temp)}.csv', index = False)